# **Predict Relief No Embedding**

Nesta etapa queremos montar um modelo que, a partir de dados de entrada, prediz se uma reclamação irá incorrer no pagamento de indenização por parte da empresa financeira.

Aqui, nossa variávl alvo é: `Company payed monetary relief` e assume o valor 1 quando a empresa fez o pagamento e 0 quando não fez.

----

# # **Importando Pickle da Base Tratada**

In [11]:
import pandas as pd

df = pd.read_pickle('./pickle/df_processed.pkl')

df.shape

(4802, 138)

---

## ## **Separar Variáveis Desnecessárias**

- Remover colunas desnecessárias ou que não podem ser usadas para a construção do modelo:
    - `Complaint embedding`
    - `Complaint ID`
    - `Date received`
    - `Sub-product`
    - `Sub-issue`
    - `Issue`
    - `ZIP code`
    - `Consumer complaint narrative`
    - `Company response to consumer`
    
- Remover colunas que possuem poucas informações ou pouca variabilidade:
    - `Tags`
    - `State`

- Remover colunas que indicam informações que só poderíamos ter após a reclamação ser resolvida:
    - `Company response to consumer`
    - `Consumer disputed?`
    - `Suggested Category /Product / Reasoning`
    - `Potential Annual Revenue`

In [12]:
# Importando bibliotecas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.impute import SimpleImputer

# Separando as colunas removidas em um DataFrame separado
removed_columns = ['Complaint embedding', 'Complaint ID', 'Company', 'Date received',
                   'Sub-product', 'Sub-issue', 'Issue', 'ZIP code',
                   'Consumer complaint narrative', 'Company response to consumer']

# Adicionando as colunas que começam com 'Tags', 'Company public response_Company', 'Consumer disputed?', 'Suggested'
removed_columns += [col for col in df.columns if col.startswith('Tags')]
removed_columns += [col for col in df.columns if col.startswith('Company public response_Company')]
removed_columns += [col for col in df.columns if col.startswith('Consumer disputed?')]
removed_columns += [col for col in df.columns if col.startswith('Suggested')]
removed_columns += [col for col in df.columns if col.startswith('State')]

# Adicionando 'Potential Annual Revenue'
removed_columns.append('Potential Annual Revenue')

# Criando o DataFrame com as colunas removidas
df_removed = df[removed_columns].copy()

# Removendo as colunas do DataFrame original
df = df.drop(columns=removed_columns)


---



# # **Modelo de Previsão de Pagamento de Indenização**

- Modelo: **Random Forest Classifier** é um modelo adquado quando a variável alvo é do tipo categórica.

In [13]:
# Definindo a variável alvo e as variáveis preditoras
y = df['Company payed monetary relief']  # Variável alvo
X = df.drop(columns=['Company payed monetary relief'])  # Variáveis preditoras

# Preencher valores ausentes com a média das colunas
#imputer = SimpleImputer(strategy='mean')
#X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Treinando um modelo RandomForest
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Previsões
y_pred = model.predict(X_test)

# Reunindo as colunas removidas ao DataFrame original
df_final = pd.concat([X_test, df_removed.loc[X_test.index]], axis=1)

# Avaliação do modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred))
print("Relatório de Classificação:\n", classification_report(y_test, y_pred))


Acurácia: 0.9077029840388618
Matriz de Confusão:
 [[635  86]
 [ 47 673]]
Relatório de Classificação:
               precision    recall  f1-score   support

       False       0.93      0.88      0.91       721
        True       0.89      0.93      0.91       720

    accuracy                           0.91      1441
   macro avg       0.91      0.91      0.91      1441
weighted avg       0.91      0.91      0.91      1441



# # **Resultados**

|                  | Predicted: False | Predicted: True | Total Real Values |
|------------------|------------------|-----------------|-------------------|
| **Actual: False** | 636              | 85              | 721               |
| **Actual: True**  | 50               | 670             | 720               |
| **Total Predictions**  | 686              | 755              | 1441              |

- True Negatives (TN): 636
    - Estes são os casos em que o modelo previu False e a classe real também era False. O modelo acertou em prever que a empresa não pagou o alívio monetário.

- False Positives (FP): 85
    - Estes são os casos em que o modelo previu True, mas a classe real era False. Aqui, o modelo previu que a empresa pagaria o alívio monetário, mas, na realidade, não pagou. Este é um erro tipo I.

- False Negatives (FN): 50
    - Estes são os casos em que o modelo previu False, mas a classe real era True. Neste caso, o modelo previu que a empresa não pagaria o alívio monetário, mas, na realidade, pagou. Este é um erro tipo II.

- True Positives (TP): 670
    - Estes são os casos em que o modelo previu True e a classe real também era True. O modelo acertou em prever que a empresa pagou o alívio monetário.

In [14]:
df_final.shape

(1441, 137)

----

# # **Exportando Modelo Treinado**

In [15]:
if hasattr(model, 'monotonic_cst'):
    print("Atributo encontrado")
else:
    print("Atributo não encontrado")


Atributo encontrado


In [16]:
import joblib

# Salvar o modelo treinado
joblib.dump(model, './model/random_forest_predict_relief.pkl')


['./model/random_forest_predict_relief.pkl']

---

# **Exportando Arquivos Gerados**

In [17]:
df_final.to_excel('./excel/df_predict_relief.xlsx', index=False)

In [18]:
df_final.to_pickle("./pickle/df_predict_relief.pkl")

In [19]:
print("Notebook Predict Relief Concluído")

Notebook Predict Relief Concluído


In [20]:
X.to_excel('./excel/X.xlsx', index=False)